## Построение социальных графов. Меры центральности
## Ирина Соколова

**Задание 1 (10 баллов)** 

Прочитайте данные из ```emails.csv```. В колонке ```message``` содержится информация о письме. Вам нужно сформировать несколько таблиц по данным для комфортной работы. 

Во-первых, у Вас имеется информация от том от когдо было отправленно письмо и кому:

* ```From``` - email отправителя
* ```X-From``` - имя отправитела 
* ```To``` - email получателя 
* ```X-To``` - имя получателя

Каждому индивидууму нужно присвоить свой уникальный ```id``` и сформировать таблицу, где по нему можно получить ```email``` и имя.

Также Вам нужно сформировать таблицу, в которой по каждому письму будет содержаться: 

* Дата
* ```id``` отправителя
* ```id``` получателей (обратитие внимание на то, что одно письмо может получать несколько человек)
* Текст

In [94]:
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [95]:
df = pd.read_csv("emails.csv")


In [96]:
emails = df["message"].values

In [97]:
EMAIL_INFO = re.compile(".+?X-FileName: .*?\n", re.DOTALL)
SENDER_EMAIL = re.compile('(?<=\nFrom: ).+?(?=\n)')
RECIPIENT_EMAILS = re.compile('(?<=\nTo: ).+?(?=\nSubject:)|(?<=\nCc: ).+?(?=\nMime:)|(?<=\nBcc: ).+?(?=\nX-From:)')
SENDER_NAME = re.compile('(?<=\nX-From: ).+?(?=\nX-To:)')
RECIPIENT_NAMES = re.compile('(?<=\nX-To: ).+?(?=\nX-cc:)|(?<=\nX-cc: ).+?(?=\nX-bcc:)|(?<=\nX-bcc: ).+?(?=\nX-Folder:)')
DATE = re.compile('(?<=\nDate: ).+?(?=\n)')

In [98]:
def extract_sender_recipient(email):
    info = re.findall(EMAIL_INFO, email)
    
 
    sender_address = re.findall(SENDER_EMAIL, info[0])
    if sender_address:
        from_address = sender_address[0].strip()
    else:
        from_address = "unknown"
    
    
    recipient_address = re.findall(RECIPIENT_EMAILS, info[0])
    rec_addresses_lst = []
    for a in recipient_address:
        if "," in a:
            lst = a.split(",")
            rec_addresses_lst.extend(lst)
        else:
            rec_addresses_lst.append(a)
    to_addresses = [addr.strip() for addr in rec_addresses_lst]
    
    

    sender_name = re.findall(SENDER_NAME, info[0])
    from_name = remove_symbols(sender_name[0])

    
    
    
    recipient_name = re.findall(RECIPIENT_NAMES, info[0])
    rec_names_lst = []
    for n in recipient_name:
        if "," in n:
            lst = n.split(",")
            rec_names_lst.extend(lst)
        else:
            rec_names_lst.append(n)
    to_names = [remove_symbols(name) for name in rec_names_lst]
    
    return (from_address, to_addresses, from_name, to_names)


    

In [99]:
def name_address_mappings(email):
    people = extract_sender_recipient(email)
    sender_dict = {people[2]: people[0]}
    recipient_dict = {people[3][i]: people[1][i] for i in range(len(people[3])) if len(people[3]) == len(people[1])}
    big_dict = {**recipient_dict, **sender_dict}
    return big_dict
        
    

In [100]:
def remove_symbols(s):
    s = re.sub("<.*?>", "", s)
    s = s.strip()
    s = re.sub('"', '', s)
    return s
    

In [101]:
dicts = []

for email in tqdm(emails[:1000]):
    d = name_address_mappings(email)
    dicts.append(d)
   

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

In [102]:
all_people_dict = {}
for d in dicts:
    all_people_dict.update(d)

In [103]:
names = []
addresses = []
for key in all_people_dict:
    names.append(key)
    addresses.append(all_people_dict[key])

In [104]:
people_df = pd.DataFrame({"id": range(len(all_people_dict)),
                          "name": names,
                          "address": addresses})

In [126]:
people_df

,id,name,address
0,0,Tim Belden,tim.belden@enron.com
1,1,Phillip K Allen,phillip.allen@enron.com
2,2,John J Lavorato,john.lavorato@enron.com
3,3,Leah Van Arsdall,leah.arsdall@enron.com
4,4,Randall L Gay,randall.gay@enron.com
5,5,Greg Piper,greg.piper@enron.com
6,6,david.l.johnson@enron.com,david.l.johnson@enron.com
7,7,John Shafer,john.shafer@enron.com
8,8,Joyce Teixeira,joyce.teixeira@enron.com
9,9,Mark Scott,mark.scott@enron.com


In [106]:
def extract_date(email):
    info = re.findall(EMAIL_INFO, email)
    date = re.findall(DATE, info[0])
    d = re.findall('(?<=\w\w\w, )\d{1,2} \w\w\w \d\d\d\d', date[0])
    return d[0]
    

In [107]:
def extract_text(email):
    m = re.match(".+?X-FileName: .*?\n(.*)", email, re.DOTALL)
    return m.group(1)

In [110]:
dates = []
texts = []
senders = []
recipients = []
for email in tqdm(emails[:1000]):
    try:
        date = extract_date(email)
        dates.append(date)
        text = extract_text(email)
        texts.append(text)
        sender_name, recipient_names = extract_sender_recipient(email)[2], extract_sender_recipient(email)[3]
        senders.append(sender_name)
        recipients.append(recipient_names)
    except KeyError:
        pass
        
    

    
    

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/ika/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/ika/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/ika/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [111]:
ids = people_df["id"].tolist()
name_id_map = dict(zip(names, ids))

In [120]:
sender_ids = []
for sender in senders:
    s_id = name_id_map[sender]
    sender_ids.append(s_id)

recipient_ids = []
for recipient in recipients:
    r_list = []
    for r in recipient:
        try:
            r_id = name_id_map[r]
            r_list.append(r_id)
        except KeyError:
            r_id = "not found"
            r_list.append(r_id)
    recipient_ids.append(r_list)
  
        

In [123]:
df_2 = pd.DataFrame({"date": dates,
                   "from": sender_ids,
                     "to": recipient_ids,
                     "text": texts})

In [125]:
df_2

,date,from,to,text
0,14 May 2001,1,[0],\nHere is our forecast\n\n
1,4 May 2001,1,[2],\nTraveling to have a business meeting takes t...
2,18 Oct 2000,1,[3],\ntest successful. way to go!!!
3,23 Oct 2000,1,[4],"\nRandy,\n\n Can you send me a schedule of the..."
4,31 Aug 2000,1,[5],\nLet's shoot for Tuesday at 11:45.
5,31 Aug 2000,1,[5],"\nGreg,\n\n How about either next Tuesday or T..."
6,22 Aug 2000,1,"[6, 7]",\nPlease cc the following distribution list wi...
7,14 Jul 2000,1,[8],\nany morning between 10 and 11:30
8,17 Oct 2000,1,[9],\n1. login: pallen pw: ke9davis\n\n I don't t...
9,16 Oct 2000,1,[10],\n---------------------- Forwarded by Phillip ...


**Задание 2 (15 баллов)** 

Сформируйте матрицу $A = (a_{ij})_{i,j = 1}^N$, где $a_{ij}$ - количество писем, которое индивид $i$ отправил индивиду $j$, $N$ - количество индивидуумов

Далее из матрицы $A$ получим симметричную матрицу, которая будет показывать нам частоту коммуникаций между каждой парой индивидов: $A+A^T$. Постройте по данной матрице не направленный граф, где узлы должны называться по именам, которые вы извлекли из ```X-From``` и ```X-To```

**Замечание.** Так как граф слишком большой, то вы можете оставить только наиболее активных участников.

**Задание 3 (25 баллов)**

Оцените важность отдельных узлов, используя следующие меры:

* Degree centrality
* Closeness centrality
* Betweenness centrality
* Eigenvector centrality

Подробнее читайте в статье: УЗЛЫ В СОЦИАЛЬНЫХ СЕТЯХ: МЕРЫ ЦЕНТРАЛЬНОСТИ И РОЛЬ В СЕТЕВЫХ ПРОЦЕССАХ

Для каждой меры центральности выведите топ пользователей

**Бонусное задание (25 баллов)** 

Выделите темы, которые обсуждаются в письмах.

Выберите одну из мер центральности и для топа пользователей по данной мере укажите, какие темы они обсуждают чаще всего и с кем.